In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install -r '/content/drive/MyDrive/LLMs_Project/requirements.txt'

In [4]:
import os
docs_path = '/content/drive/MyDrive/LLMs_Project/docs/'
#docs_path = '/docs/'
texts_path = '/content/drive/MyDrive/LLMs_Project/texts_extracted/'
#texts_path = '/texts_extracted/'
requirements_path = '/content/drive/MyDrive/LLMs_Project/requirements.txt'
print("Docs to be processed")
file_list = os.listdir(docs_path)
document_count = len(file_list)
print(file_list)
print(f"Total documents found: {document_count}")

Docs to be processed
['Laws of the Game 2025_26_single pages.pdf', 'FIFA Disciplinary Code_September 2025 edition_EN.pdf', 'FIFA Equipment Regulations_2025_EN.pdf', 'FWC26_Competition Regulations_EN.pdf']
Total documents found: 4


In [5]:
import pymupdf
import re
import spacy
from glob import glob
from langchain_text_splitters import SpacyTextSplitter

def clean_text(text: str) -> str:
    # Remove non-printable control characters
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def extract_text(file_path: str, destination_folder: str) -> str:
    text_lines = []
    print(f"Processing file: {file_path}")
    doc = pymupdf.open(file_path)
    for page in doc:
      text = page.get_text()
      text_lines.extend(text.splitlines())
      text_lines = [clean_text(line) for line in text_lines]
    with open(destination_folder + file_path.split("/")[-1].split(".")[0] + ".txt", "w") as file:
        file.write("\n".join(text_lines))
        print(f"Extracted text saved in: {destination_folder + file_path.split("/")[-1].split(".")[0] + '.txt'}")


def chunk_text(text: str, chunk_size=500, chunk_overlap=100):
    text_splitter = SpacyTextSplitter(
        pipeline="en_core_web_sm",  # Uses spaCy for sentence splitting
        chunk_size=chunk_size,      # Then groups sentences into chunks of this size
        chunk_overlap=chunk_overlap # Adds overlap between chunks
    )
    chunks = text_splitter.split_text(text)
    print(f"Number of chunks in: {len(chunks)}")
    return chunks

In [6]:
for file_path in glob(docs_path + "*.pdf", recursive=True):
    extract_text(file_path, texts_path)

Processing file: /content/drive/MyDrive/LLMs_Project/docs/Laws of the Game 2025_26_single pages.pdf
Extracted text saved in: /content/drive/MyDrive/LLMs_Project/texts_extracted/Laws of the Game 2025_26_single pages.txt
Processing file: /content/drive/MyDrive/LLMs_Project/docs/FIFA Disciplinary Code_September 2025 edition_EN.pdf
Extracted text saved in: /content/drive/MyDrive/LLMs_Project/texts_extracted/FIFA Disciplinary Code_September 2025 edition_EN.txt
Processing file: /content/drive/MyDrive/LLMs_Project/docs/FIFA Equipment Regulations_2025_EN.pdf
Extracted text saved in: /content/drive/MyDrive/LLMs_Project/texts_extracted/FIFA Equipment Regulations_2025_EN.txt
Processing file: /content/drive/MyDrive/LLMs_Project/docs/FWC26_Competition Regulations_EN.pdf
Extracted text saved in: /content/drive/MyDrive/LLMs_Project/texts_extracted/FWC26_Competition Regulations_EN.txt


In [7]:
chunks = []
for file_path in glob(texts_path + "*.txt", recursive=True):
    with open(file_path, "r") as file:
        text = file.read()
    content = chunk_text(text)
    metadata = {"source": file_path.split("/")[-1].split(".")[0]}
    for chunk in content:
        chunks.append({"chunk": chunk, "metadata": metadata})
print(f"Total number of chunks created: {len(chunks)}")

/usr/local/lib/python3.12/dist-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Number of chunks in: 334


Number of chunks in: 466


Number of chunks in: 234


Number of chunks in: 284
Total number of chunks created: 1318


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

In [ ]:
# 0 - 334 FIFA Equipment Regulations_2025_EN
# 334 - 800 Laws of the Game 25/26
# 800 - 1034 FIFA Disciplinary Code
# 1034 - 1318 FWC26_Competition Regulations_EN

In [ ]:
texts = [c["chunk"] for c in chunks]

284


In [ ]:
embeddings = []
for i, text in enumerate(texts):
    print(f"Processing chunk: {i+1}/{len(texts)}")
    embedding = model.encode(text, show_progress_bar=True)
    embeddings.append(embedding)

Processing chunk: 1/284


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing chunk: 2/284


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing chunk: 3/284


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing chunk: 283/284


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing chunk: 284/284


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
print(len(embeddings))
print(len(embeddings[0]))

284
1024


In [ ]:
import json
with open('/content/drive/MyDrive/LLMs_Project/chunks.json', 'w') as fout:
    json.dump(chunks, fout)

In [51]:
import numpy as np
np.save("/content/drive/MyDrive/LLMs_Project/embedding.npy", embeddings)